In [333]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import os
import sys
import numpy as np
import rasterio
from rasterio import plot
import datetime
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox , Geometry,CustomUrlParam
from shapely.geometry import Polygon
from IPython.display import Javascript, display
from ipywidgets import widgets,Layout
from PIL import Image

# def run_all(ev):
#     display(Javascript('IPython.notebook.execute_cells_below()'))

#from osgeo import gdal

In [334]:
# INSTANCE_ID = '4e90288f-9ccd-4841-9ae5-db857a41c493'

In [335]:
from sentinelhub import SHConfig


INSTANCE_ID = '74b3a3d5-14d7-4af0-b08c-d579e0ff2c21'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

# Crop Monitoring Application

In [336]:
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))

    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

### Enter your Top-Left and Bottom-right coordinates here.

In [337]:

top_left_x = widgets.FloatText(
    #value=-122.194405,
    description='X:',
    disabled=False
)
top_left_y = widgets.FloatText(
    #value=39.624598,
    description='Y:',
    disabled=False
)
bottom_right_x = widgets.FloatText(
    #value=-122.117157,
    description='X:',
    disabled=False
)
bottom_right_y = widgets.FloatText(
    #value=39.583069,
    description='Y:',
    disabled=False
)

#### 1. Top-Left Coordinates:

In [338]:
top_left_x

FloatText(value=0.0, description='X:')

In [339]:
top_left_y

FloatText(value=0.0, description='Y:')

#### 2. Bottom-right Coordinates:

In [340]:
bottom_right_x

FloatText(value=0.0, description='X:')

In [342]:
bottom_right_y

FloatText(value=39.583069, description='Y:')

In [344]:
# print(top_left_x.value)
# print(top_left_y.value)
# print(bottom_right_y.value)
# print(bottom_right_x.value)

In [345]:
out_pl = widgets.Output()

#### Sample coordinates:
1.  chico_coords_wgs84 = [-122.194405, 39.624598, -122.117157, 39.583069]
2.  betsiboka_coords_wgs84 = [80.209067, 21.369876, 80.215933, 21.359038]
3.  betsiboka_coords_wgs842 = [46.16, -16.15, 47.01, -15.58]



In [346]:
#input_coords_wgs84 = [top_left_x.value,top_left_y.value,bottom_right_x.value, bottom_right_y.value]
# betsiboka_coords_wgs84 = [46.16, -16.15, 47.01, -15.58]


# chico_coords_wgs84 = [-122.194405, 39.624598, -122.117157, 39.583069]
# betsiboka_coords_wgs84 = [80.209067, 21.369876, 80.215933, 21.359038]
# betsiboka_coords_wgs84 = [46.16, -16.15, 47.01, -15.58]
# betsiboka_coords_wgs84 = [-122.194405, 39.624598, -122.117157, 39.583069]
#betsiboka_coords_wgs84 = [1292344.0 ,5205055.5 ,1310615.0 ,5205055.5]

In [347]:
#input_bbox = BBox(bbox=input_coords_wgs84, crs=CRS.WGS84)

In [348]:
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox

def Run_this(change):
    
    
    input_coords_wgs84 = [top_left_x.value,top_left_y.value,bottom_right_x.value, bottom_right_y.value]
    input_bbox = BBox(bbox=input_coords_wgs84, crs=CRS.WGS84)
    
    print(input_coords_wgs84)
    
    wms_true_color_request = WmsRequest(
        layer='TRUE-COLOR-S2-L1C',
        bbox=input_bbox,
        time=('2017-12-01', '2017-12-31'),
        width=512, height=856,
        maxcc=0.3,
        config=config 
    )
    wms_bands_request = WcsRequest(
    layer='BANDS-S2-L1C',
    bbox=input_bbox,
    time='latest',
    resx='10m',
    resy='10m',
    maxcc=0.1,
    image_format=MimeType.TIFF_d32f,
    config=config,
    #custom_url_params={
    #    CustomUrlParam.GEOMETRY: "POLYGON((13.522606 -86.160791,13.522462 -86.160989,13.522387 -86.161045,13.522316 -86.161049,13.522236 -86.16109,13.522079 -86.161024,13.521843 -86.161166,13.521601 -86.161072,13.521576 -86.161009,13.52146 -86.16105,13.521368 -86.160921,13.521312 -86.160904,13.521313 -86.160778,13.521438 -86.160657,13.521674 -86.160649,13.521895 -86.160569,13.522006 -86.160512,13.522087 -86.160532,13.522288 -86.1606,13.522346 -86.160557,13.522391 -86.160526,13.522522 -86.160499,13.522622 -86.160463,13.522696 -86.160418,13.522643 -86.160602,13.522608 -86.160696,13.522606 -86.160791))"
    #}
    )
    
    wms_bands_img = wms_bands_request.get_data()
    img = wms_bands_img[-1][:, :, [3, 2, 1]]


    
    red = np.array(wms_bands_img[-1][:, :, 3])
    nir = np.array(wms_bands_img[-1][:, :, 7])
    ndvi = np.divide(np.subtract(red, nir), np.add(red, nir))
    
    b = np.where(np.isnan(ndvi), 0, ndvi)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20,20))
    image = wms_bands_img[-1][:, :, [3, 2, 1]]
    factor = 1
    if np.issubdtype(image.dtype, np.floating):
        axs[0].imshow(np.minimum(image * factor, 1))
        axs[0].set_title('Actual Image')
    else:
        axs[0].imshow(image)
        axs[0].set_title('Actual Image')

    axs[1].imshow(b, cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green","yellow","red"])) # Typically the color map for NDVI maps are the Red to Yellow to Green
    axs[1].set_title('NDVI')
    
    fig = plt.gcf()
    fig.set_size_inches(20,20)
    fig.savefig("full_figure.png", bbox_inches='tight')
    plt.close(fig)
    
    image_final = Image.open('full_figure.png')
    
    out_pl.clear_output()
    
    with out_pl: 
        display(image_final)

In [349]:
#Run_this()

In [350]:
button = widgets.Button(description="Run", 
                        layout=Layout(width='50%', height='80px'), button_style = 'info')
button.on_click(Run_this)


In [351]:
#Run_this(1)

In [352]:
widgets.VBox([button, out_pl])


[-122.194405, 39.624598, -122.117157, 39.583069]


In [102]:
#wms_bands_img[0].shape

In [331]:

# plt.show()
    
#plot_image(wms_bands_img[-1][:, :, [3, 2, 1]])

In [105]:
# fig = plt.figure(figsize=(10, 10))
# fig.set_facecolor('white')
# plt.imshow(b, cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green","yellow","red"])) # Typically the color map for NDVI maps are the Red to Yellow to Green
# plt.title('NDVI')
# plt.show()